In [1]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


<ipython-input-1-5cfb607d7a2e>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import urllib3

In [3]:
http = urllib3.PoolManager()

In [4]:
from config_batch import * 

# Functions

In [5]:
ws_hostname = "127.0.1.1"
ws_hostname = "10.1.0.45"

# ws_hostname = "192.168.1.3"


In [6]:
def call_ws(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    params = urllib.parse.urlencode({"street": addr_data[street_field],
                                     "housenumber": addr_data[housenbr_field],
                                     "city": addr_data[city_field],
                                     "postcode": addr_data[postcode_field],
                                     "country": addr_data[country_field],
                                     "check_result" : "yes" if check_result else "no",
                                     "struct_osm" : "yes" if structured_osm else "no"
                                    })
    url = f"http://{ws_hostname}:5000/search/?{params}"
#     print(url)
    try:
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             print(res)
            res["time"] = datetime.now() - t
            return res
    except Exception as e:
        return str(e)
    

In [25]:
def call_ws_batch(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
#     print(addr_data)
#     print(addr_data.shape)
#     print()
    file_data = addr_data.rename(columns = {
        street_field : "street",
        housenbr_field: "housenumber",
        postcode_field: "postcode",
        city_field: "city",
        country_field: "country",
        addr_key_field : "addr_key"
    }).to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "with_rejected" : "yes" if with_reject else "no",
        "check_result" : "yes" if check_result else "no",
        "struct_osm" : "yes" if structured_osm else "no",
        #"extra_house_nbr": "no"
    })
    
    try:
        res = pd.DataFrame(json.loads(r.data.decode('utf-8')))
    except ValueError:
        print("Cannot decode result:")
        print(r.data.decode('utf-8'))
        print(json.loads(r.data.decode('utf-8')))
        return 
#     display(res)
    return res

In [8]:
def expand_json(addresses):
    addresses["status"]= addresses.json.apply(lambda d: "error" if "error" in d else "match" if "match" in d else "rejected")
    addresses["time"]  = addresses.json.apply(lambda d: d["time"])

    addresses["timing"]  = addresses.json.apply(lambda d: d["timing"] if "timing" in d else {})

    addresses["method"]= addresses.json.apply(lambda d: d["match"][0]["method"] if len(d)>0 and "match" in d else "none")
    
    for field in ["street", "number", "postcode", "city"]:
        addresses[field]= addresses.json.apply(lambda d: d["match"][0]["addr_out_"+field] if len(d)>0 and "match" in d else "")
    return 

# Calls

## Single address calls

In [9]:
call_ws({street_field:   "Av. Fonsny", 
         housenbr_field: "20 bus 22",
         city_field:     "Saint-Gilles",
         postcode_field: "1060",
         country_field:  "Belgium"}, check_result=True, structured_osm=False)

{'match': [{'SIM_city': 0.46153846153846156,
   'SIM_house_nbr': 0.0,
   'SIM_street': 1.0,
   'SIM_street_which': 'addr_out_street',
   'SIM_zip': 1.0,
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': 'DAE (Smals), 20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
   'in_house_nbr': '20 bus 22',
   'lat': '50.8358216',
   'lon': '4.3386884',
   'lpost_house_nbr': '20',
   'lpost_unit': 'bus 22',
   'method': 'orig',
   'osm_addr_in': 'Av. Fonsny, 20 bus 22, 1060 Saint-Gilles, Belgium',
   'place_id': 173261,
   'place_rank': 30}],
 'rejected': [{'SIM_city': 0.46153846153846156,
   'SIM_house_nbr': 0.0,
   'SIM_street': 1.0,
   'SIM_street_which': 'addr_out

In [ ]:
call_ws({street_field:   "", 
         housenbr_field: "",
         city_field:     "Dinant",
         postcode_field: "5500",
         country_field:  "Belgium"}, check_result=True, structured_osm=True)

## Batch calls (row by row)

In [136]:
addresses = get_addresses("address.csv.gz")
addresses = addresses.sample(1000).copy()

### Simple way

In [149]:
addresses["json"] = addresses.progress_apply(call_ws, check_result=False, structured_osm=False, axis=1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
# addresses


### Using Dask

In [ ]:
dd_addresses = dd.from_pandas(addresses, npartitions=4)

dask_task = dd_addresses.apply(call_ws, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses["json"] = dask_task.compute()

In [ ]:
expand_json(addresses)

In [ ]:
addresses

## Batch calls (batch WS)

In [10]:
addresses = pd.read_csv(f"../GISAnalytics/data/geocoding/kbo_1000_sample.csv")
addresses = addresses.rename(columns={"Unnamed: 0": addr_key_field, "address": street_field})
addresses[city_field] = ""
addresses[country_field] =  "Belgique"
addresses[housenbr_field] = ""
addresses[postcode_field]=""
addresses

,EntityNumber,StreetFR,MunicipalityFR,CountryFR,HouseNumber,Zipcode
0,2762939,"Kriekenlaan, 22, 2240 Zandhoven, Belgique",,Belgique,,
1,2253494,"Cardijnlaan(D), *, 8600 Diksmuide, Belgique",,Belgique,,
2,1764140,"Poststraat, 19, 9860 Oosterzele, Belgique",,Belgique,,
3,87070,"Sylvain Dupuisstraat, 26, 8300 Knokke-Heist, B...",,Belgique,,
4,2261339,"Oude Baan(M), 338, 3630 Maasmechelen, Belgique",,Belgique,,
...,...,...,...,...,...,...
995,2720764,"Rue du Wainage(VEL), 62, 5060 Sambreville, Bel...",,Belgique,,
996,386196,"Chaussée de Tournai(R-C), 4, 7520 Tournai, Bel...",,Belgique,,
997,1474962,"Rue Louis Caty(B), 32, 7331 Saint-Ghislain, Be...",,Belgique,,
998,2424889,"Drève des Bouleaux, 5, 7090 Braine-le-Comte, B...",,Belgique,,


In [137]:
addresses

,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
1693620,2.006.998.987,Belgique,9750,Kruisem,Alfred Amelotstraat,22
2245463,2.159.185.851,Belgique,8790,Waregem,Kalkhoevestraat,1
641094,0627.942.168,Belgique,1080,Molenbeek-Saint-Jean,Avenue du Condor,19
1896185,2.064.435.063,Belgique,8800,Roeselare,Noordstraat,197
2729264,2.241.540.138,Belgique,2990,Wuustwezel,Bredabaan (Gooreind),23/4
...,...,...,...,...,...,...
3097222,2.285.822.024,Belgique,5030,Gembloux,"Rue Camille-Cals, Ernage",28
180091,0434.204.464,Belgique,1180,Uccle,Rue du Roetaert,40
2524886,2.210.605.452,Belgique,1020,Bruxelles,Rue Marie-Christine,153
2520632,2.209.879.437,Belgique,1080,Molenbeek-Saint-Jean,Rue Picard,57


### Single block

In [26]:
# Only geocoding
# addresses["StreetFR"] = ""
call_ws_batch(addresses, mode="geo", check_result=True, structured_osm=True)

,addr_key,lat,lon,place_rank,method
0,1002214,50.821627,3.168548,30.0,libpostal+regex[lpost]+photon
1,1003571,51.130720,4.296507,26.0,libpostal+regex[lpost]+photon
2,1011436,50.695095,4.676402,30.0,libpostal+regex[lpost]+photon
3,1011475,50.819275,4.386580,30.0,libpostal+regex[lpost]+photon
4,1018679,51.003970,5.047784,30.0,libpostal+regex[lpost]+photon
...,...,...,...,...,...
980,3075050,50.613286,5.940212,26.0,photon
981,558959,50.763989,3.730486,26.0,photon
982,647118,50.846113,4.436441,26.0,photon
983,940484,50.348816,4.857279,26.0,photon


In [144]:
# Geocode + address
call_ws_batch(addresses, mode="short") 

,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,in_house_nbr,lpost_house_nbr,lpost_unit,addr_out_postcode,addr_out_city,addr_out_country
0,2762939,51.211615,4.649820,30.0,libpostal+regex[lpost]+photon,992278.0,Kriekenlaan,22,None,22,,2240,Zandhoven,België / Belgique / Belgien
1,2253494,51.022370,2.860610,26.0,libpostal+regex[lpost]+photon,289768.0,Cardijnlaan,None,None,,,8600,Diksmuide,België / Belgique / Belgien
2,1764140,50.924678,3.788056,26.0,libpostal+regex[lpost]+photon,413714.0,Poststraat,None,None,19,,9860,Balegem,België / Belgique / Belgien
3,87070,51.351599,3.286075,26.0,libpostal+regex[lpost]+photon,403430.0,Sylvain Dupuisstraat,None,None,26,,8300,Knokke-Heist,België / Belgique / Belgien
4,2261339,50.977155,5.690444,26.0,libpostal+regex[lpost]+photon,387798.0,Oude Baan,None,None,,m 338,3630,Maasmechelen,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2720764,50.462361,4.596458,30.0,libpostal+regex[lpost]+photon,2624288.0,Rue du Wainage,62,None,62,,5060,Sambreville,België / Belgique / Belgien
996,386196,50.561071,3.424308,30.0,libpostal+regex[lpost]+photon,3530216.0,Chaussée de Tournai,4,None,4,,7641,Bruyelle,België / Belgique / Belgien
997,1474962,50.479737,3.838976,26.0,libpostal+regex[lpost]+photon,418941.0,Rue Louis Caty,None,None,32,,7331,Saint-Ghislain,België / Belgique / Belgien
998,2424889,50.606398,4.159495,26.0,libpostal+regex[lpost]+photon,313717.0,Drève des Bouleaux,None,None,5,,7090,Braine-Le-Comte,België / Belgique / Belgien


In [ ]:
# Geocode + address, with rejected addresses
call_ws_batch(addresses, mode="long", with_reject=True)

In [ ]:
# call_ws_batch(addresses[addresses.EntityNumber.str.startswith("0554.81")], mode="long", with_reject=True)

In [ ]:
# a[a.in_house_nbr.str.upper() != a.lpost_house_nbr.str.upper()]

### Batch blocs

In [ ]:
def call_ws_batch_chunks(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False, chunk_size=100): 
    ## TODO : find a better way with dask? It seems that map_partitions does not support function returning dataframes. 

    chunks = np.array_split(addr_data, addr_data.shape[0]//chunk_size)

    res= [call_ws_batch(chunk, mode=mode, 
                        check_result=check_result, 
                        structured_osm=structured_osm) for chunk in tqdm(chunks)]
    df_res = pd.concat(res, sort=False)
    return df_res

In [146]:
df_res = call_ws_batch_chunks(addresses, chunk_size=100, mode="short", check_result=False)
df_res

  0%|          | 0/10 [00:00<?, ?it/s]

,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,in_house_nbr,lpost_house_nbr,lpost_unit,addr_out_postcode,addr_out_city,addr_out_country
0,2762939,51.211615,4.649820,30.0,libpostal+regex[lpost]+photon,992278.0,Kriekenlaan,22,None,22,,2240,Zandhoven,België / Belgique / Belgien
1,2253494,51.022370,2.860610,26.0,libpostal+regex[lpost]+photon,289768.0,Cardijnlaan,None,None,,,8600,Diksmuide,België / Belgique / Belgien
2,1764140,50.924678,3.788056,26.0,libpostal+regex[lpost]+photon,413714.0,Poststraat,None,None,19,,9860,Balegem,België / Belgique / Belgien
3,87070,51.351599,3.286075,26.0,libpostal+regex[lpost]+photon,403430.0,Sylvain Dupuisstraat,None,None,26,,8300,Knokke-Heist,België / Belgique / Belgien
4,2261339,50.977155,5.690444,26.0,libpostal+regex[lpost]+photon,387798.0,Oude Baan,None,None,,m 338,3630,Maasmechelen,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2720764,50.462361,4.596458,30.0,libpostal+regex[lpost]+photon,2624288.0,Rue du Wainage,62,None,62,,5060,Sambreville,België / Belgique / Belgien
96,386196,50.648278,3.343255,26.0,libpostal+regex[lpost]+photon,307555.0,Chaussée de Tournai,None,None,4,,7520,Ramegnies-Chin,België / Belgique / Belgien
97,1474962,50.479737,3.838976,26.0,libpostal+regex[lpost]+photon,418941.0,Rue Louis Caty,None,None,32,,7331,Saint-Ghislain,België / Belgique / Belgien
98,2424889,50.606398,4.159495,26.0,libpostal+regex[lpost]+photon,313717.0,Drève des Bouleaux,None,None,5,,7090,Braine-Le-Comte,België / Belgique / Belgien


In [129]:
df_res[df_res.method=="nonum"].sort_values("postcode")


,addr_key,country,postcode,city,street,housenumber,osm_addr_in,place_id,lat,lon,...,addr_out_city,addr_out_number,addr_out_country,addr_out_postcode,addr_out_other,retry_on_26,method,in_house_nbr,lpost_house_nbr,lpost_unit
54,0413.300.172,Belgique,1040,Etterbeek,Rue Gray,96-102,"Rue Gray, , 1040 Etterbeek, Belgique",993722.0,50.834601,4.379528,...,Etterbeek,None,België / Belgique / Belgien,1040,Région de Bruxelles-Capitale - Brussels Hoofds...,None,nonum,96-102,96-102,
26,2.268.632.238,Belgique,1490,Court-Saint-Etienne,Rue de Limauge,27B,"Rue de Limauge, , 1490 Court-Saint-Etienne, Be...",1227319.0,50.652189,4.531798,...,Court-Saint-Étienne,None,België / Belgique / Belgien,1341,Wallonie,None,nonum,27B,27b,
19,0873.101.156,Belgique,2870,Puurs,Stationsstraat,10EA,"Stationsstraat, , 2870 Puurs, Belgique",275030.0,51.077196,4.281867,...,Puurs-Sint-Amands,None,België / Belgique / Belgien,2870,Vlaanderen,None,nonum,10EA,10ea,
88,0418.246.380,Belgique,6040,Charleroi,Allée Verte,SN,"Allée Verte, , 6040 Charleroi, Belgique",438902.0,50.430774,4.413761,...,Jumet,None,België / Belgique / Belgien,6040,Wallonie,None,nonum,SN,sn,
37,2.225.645.204,Belgique,7050,Jurbise,Chemin des Vachers(EL),8,"Chemin des Vachers(EL), , 7050 Jurbise, Belgique",533291.0,50.511001,3.921121,...,Jurbise,None,België / Belgique / Belgien,7050,Jurbise - Wallonie,None,nonum,8,8,
17,0713.751.734,Belgique,9820,Merelbeke,Guldensporenpark,2 Gebouw A,"Guldensporenpark, , 9820 Merelbeke, Belgique",764489.0,51.004196,3.754740,...,Merelbeke,None,België / Belgique / Belgien,9820,Vlaanderen,None,nonum,2 Gebouw A,2,


In [ ]:
df_res[df_res.in_house_nbr.str.upper() != df_res.lpost_house_nbr.str.upper()]

In [ ]:
# df_res[df_res.addr_out_number.str.upper() != df_res.lpost_house_nbr.str.upper()]

In [ ]:
df_res.method.value_counts()

## Comparing options

In [ ]:
addresses = get_addresses("address.csv.gz")
addresses = addresses[addresses[country_field] == "Belgique"]
addresses = addresses.sample(10000).copy()

In [ ]:
results = {}
it_per_seconds=pd.DataFrame()

for check_label in ["check", "nocheck"]:
    for struct_label in ["struct", "unstruct" ]:
        print(check_label, struct_label)
        start=datetime.now()
        
        results[(check_label, struct_label)] = call_ws_batch_chunks(addresses, 
                                                                    mode="short", 
                                                                    check_result   =  check_label == "check", 
                                                                    structured_osm =  struct_label == "struct")
        
        it_per_seconds.loc[check_label, struct_label] = addresses.shape[0] / (datetime.now()-start).total_seconds()
print("Iterations per seconds:")
it_per_seconds

In [ ]:
print("Match rate")
pd.DataFrame({k1: {k2: results[(k1,k2)].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Match rate (without nostreet)")
pd.DataFrame({k1: {k2: results[(k1,k2)].query("method!='nostreet'").shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Unmatched addresses")
for k1 in results:
    print(k1)
    nomatch=addresses[~addresses[addr_key_field].isin(results[k1]["addr_key"])]
    display(nomatch)
    print(nomatch[country_field].value_counts())

In [ ]:
vc_values = pd.DataFrame(columns=results.keys(), index=results.keys())

for k1 in results:
    vc_values.loc[k1, k1] = results[k1].shape[0]
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
 
            vc = mg._merge.value_counts()

            mismatches = mg[mg.place_id_x != mg.place_id_y][["addr_key"]]
            mismatches = mismatches.merge(addresses.rename({addr_key_field:"addr_key"}, axis=1))
            mismatches = mismatches.merge(r1[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches = mismatches.merge(r2[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches.columns = pd.MultiIndex.from_arrays([["Input"]*6 + [f"x:{k1}"]*5 + [f"y:{k2}"]*5, mismatches.columns])

            mismatch_values = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).fillna("") != 
                                          mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).fillna("")).any(axis=1)]
            
            mismatch_values_no_nmbr = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).drop("addr_out_number", axis=1).fillna("") != 
                                                  mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).drop("addr_out_number", axis=1).fillna("")).any(axis=1)]
            
            
            vc_label = f"{vc['both']} ({mismatches.shape[0]} - {mismatch_values.shape[0]} - {mismatch_values_no_nmbr.shape[0]}) / {vc['left_only']} / {vc['right_only']}"
            vc_values.loc[k1, k2]=vc_label

                
            print(f"{k1} vs {k2}")
            print(vc_label)
            print("-----------------------------")
            
            print(f"Only in {k1}")
            display(r1[r1.addr_key.isin(mg[mg._merge=="left_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print(f"Only in {k2}")
            display(r2[r2.addr_key.isin(mg[mg._merge=="right_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print("Mismatch on place_id")
            display(mismatches)
            
            print("Mismatch on values")
            
            display(mismatch_values)
            
            print("Mismatch on values (no nbr)")
            display(mismatch_values_no_nmbr)
            
            print("#######################")
            
# display(vc_values.fillna(""))

In [ ]:
print("Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns")
vc_values.fillna("")